In [56]:
import pymongo
import pandas as pd
from pymongo import MongoClient
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

In [57]:
def data_extraction(url,dbname,collectname):
    client = MongoClient()
    client = MongoClient(url)
    db = client[dbname]
    test = db[collectname]
    df = pd.DataFrame(list(test.find()))
    return df

In [62]:
LCOD_df=data_extraction("mongodb://localhost:27017/",'DAP','LCOD')

In [63]:
LCOD_df.shape

(10868, 6)

In [64]:
LCOD_df=LCOD_df.drop_duplicates()
LCOD_df.shape

(10868, 6)

In [65]:
LCOD_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10868 entries, 0 to 10867
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   _id              10868 non-null  object
 1   _113_cause_name  10868 non-null  object
 2   cause_name       10868 non-null  object
 3   state            10868 non-null  object
 4   deaths           10868 non-null  object
 5   aadr             10868 non-null  object
dtypes: object(6)
memory usage: 594.3+ KB


In [66]:
LCOD_df.head()

,_id,_113_cause_name,cause_name,state,deaths,aadr
0,63a0a3c05865b8927e89e882,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,United States,169936,49.4
1,63a0a3c05865b8927e89e883,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Alabama,2703,53.8
2,63a0a3c05865b8927e89e884,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Alaska,436,63.7
3,63a0a3c05865b8927e89e885,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Arizona,4184,56.2
4,63a0a3c05865b8927e89e886,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Arkansas,1625,51.8


In [73]:
mean_value=LCOD_df['_113_cause_name'].mode()
LCOD_df['_113_cause_name'].fillna(value=mean_value, inplace=True)

mean_value=LCOD_df['cause_name'].mode()
LCOD_df['cause_name'].fillna(value=mean_value, inplace=True)

mean_value=LCOD_df['state'].mode()
LCOD_df['state'].fillna(value=mean_value, inplace=True)

mean_value=LCOD_df['deaths'].mean()
LCOD_df['deaths'].fillna(value=mean_value, inplace=True)

mean_value=LCOD_df['aadr'].astype(float).mode()
LCOD_df['aadr'].fillna(value=mean_value, inplace=True)


In [74]:
LCOD_df.isna().sum()

_id                0
_113_cause_name    0
cause_name         0
state              0
deaths             0
aadr               0
dtype: int64

In [76]:

#establishing the connection
conn = psycopg2.connect(
   database="postgres", user='postgres', password='rajat@123', host='127.0.0.1', port= '5432'
)
conn.autocommit = True

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

# #Preparing query to create a database
sql = '''CREATE database DAP_NEW''';

#Creating a database
cursor.execute(sql)
print("Database created successfully........")

#Closing the connection
#conn.close()

Database created successfully........


In [77]:
conn=psycopg2.connect(database="dap_new",user="postgres",password="rajat@123",host='127.0.0.1',port='5432')

In [78]:
conn.autocommit = True
cursor = conn.cursor()

In [80]:
LCOD_df.columns

Index(['_id', '_113_cause_name', 'cause_name', 'state', 'deaths', 'aadr'], dtype='object')

In [82]:
LCOD_df.head(1)

,_id,_113_cause_name,cause_name,state,deaths,aadr
0,63a0a3c05865b8927e89e882,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,United States,169936,49.4


In [92]:
LCOD_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10868 entries, 0 to 10867
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   _id              10868 non-null  object
 1   _113_cause_name  10868 non-null  object
 2   cause_name       10868 non-null  object
 3   state            10868 non-null  object
 4   deaths           10868 non-null  object
 5   aadr             10868 non-null  object
dtypes: object(6)
memory usage: 594.3+ KB


In [84]:
cursor.execute("DROP TABLE IF EXISTS lcod")
cursor.execute('''CREATE TABLE lcod(
   _id VARCHAR(50) NOT NULL,
   _113_cause_name CHAR(100),
   cause_name CHAR(100),
   state CHAR(50),
   deaths FLOAT(20),aadr FLOAT(20)
)''')
print("Table created successfully........")
conn.commit()
#Closing the connection
#conn.close()

Table created successfully........


In [97]:
LCOD_df['_113_cause_name'].value_counts()

Accidents (unintentional injuries) (V01-X59,Y85-Y86)                     988
All Causes                                                               988
Alzheimer's disease (G30)                                                988
Cerebrovascular diseases (I60-I69)                                       988
Chronic lower respiratory diseases (J40-J47)                             988
Diabetes mellitus (E10-E14)                                              988
Diseases of heart (I00-I09,I11,I13,I20-I51)                              988
Influenza and pneumonia (J09-J18)                                        988
Intentional self-harm (suicide) (*U03,X60-X84,Y87.0)                     988
Malignant neoplasms (C00-C97)                                            988
Nephritis, nephrotic syndrome and nephrosis (N00-N07,N17-N19,N25-N27)    988
Name: _113_cause_name, dtype: int64

In [102]:
LCOD_df['_113_cause_name'] = LCOD_df['_113_cause_name'].str.replace('\'s', '')
LCOD_df['cause_name'] = LCOD_df['cause_name'].str.replace('\'s', '')

In [103]:
LCOD_df['_113_cause_name'].value_counts()

Accidents (unintentional injuries) (V01-X59,Y85-Y86)                     988
All Causes                                                               988
Alzheimer disease (G30)                                                  988
Cerebrovascular diseases (I60-I69)                                       988
Chronic lower respiratory diseases (J40-J47)                             988
Diabetes mellitus (E10-E14)                                              988
Diseases of heart (I00-I09,I11,I13,I20-I51)                              988
Influenza and pneumonia (J09-J18)                                        988
Intentional self-harm (suicide) (*U03,X60-X84,Y87.0)                     988
Malignant neoplasms (C00-C97)                                            988
Nephritis, nephrotic syndrome and nephrosis (N00-N07,N17-N19,N25-N27)    988
Name: _113_cause_name, dtype: int64

In [104]:
count=0
for row in LCOD_df.itertuples():
    count=count+1
    #print(row)
    #print(str(count)+"==>"+str(row))
    print(row[1],str(row[2]),row[3],row[4],row[5],row[6])
    cursor.execute("INSERT INTO public.lcod(_id, _113_cause_name, cause_name, state, deaths, aadr)  VALUES ('"+str(row[1])+"','"+str(row[2])+"','"+str(row[3])+"','"+str(row[4])+"','"+str(row[5])+"','"+str(row[6])+"');")

63a0a3c05865b8927e89e882 Accidents (unintentional injuries) (V01-X59,Y85-Y86) Unintentional injuries United States 169936 49.4
63a0a3c05865b8927e89e883 Accidents (unintentional injuries) (V01-X59,Y85-Y86) Unintentional injuries Alabama 2703 53.8
63a0a3c05865b8927e89e884 Accidents (unintentional injuries) (V01-X59,Y85-Y86) Unintentional injuries Alaska 436 63.7
63a0a3c05865b8927e89e885 Accidents (unintentional injuries) (V01-X59,Y85-Y86) Unintentional injuries Arizona 4184 56.2
63a0a3c05865b8927e89e886 Accidents (unintentional injuries) (V01-X59,Y85-Y86) Unintentional injuries Arkansas 1625 51.8
63a0a3c05865b8927e89e887 Accidents (unintentional injuries) (V01-X59,Y85-Y86) Unintentional injuries California 13840 33.2
63a0a3c05865b8927e89e888 Accidents (unintentional injuries) (V01-X59,Y85-Y86) Unintentional injuries Colorado 3037 53.6
63a0a3c05865b8927e89e889 Accidents (unintentional injuries) (V01-X59,Y85-Y86) Unintentional injuries Connecticut 2078 53.2
63a0a3c05865b8927e89e88a Acciden

63a0a3c05865b8927e89e9a4 Diabetes mellitus (E10-E14) Diabetes New Hampshire 340 19.2
63a0a3c05865b8927e89e9a5 Diabetes mellitus (E10-E14) Diabetes New Jersey 1908 16.9
63a0a3c05865b8927e89e9a6 Diabetes mellitus (E10-E14) Diabetes New Mexico 673 26.5
63a0a3c05865b8927e89e9a7 Diabetes mellitus (E10-E14) Diabetes New York 4176 16.8
63a0a3c05865b8927e89e9a8 Diabetes mellitus (E10-E14) Diabetes North Carolina 2903 23.6
63a0a3c05865b8927e89e9a9 Diabetes mellitus (E10-E14) Diabetes North Dakota 194 21.8
63a0a3c05865b8927e89e9aa Diabetes mellitus (E10-E14) Diabetes Ohio 3740 25.2
63a0a3c05865b8927e89e9ab Diabetes mellitus (E10-E14) Diabetes Oklahoma 1398 30.6
63a0a3c05865b8927e89e9ac Diabetes mellitus (E10-E14) Diabetes Oregon 1243 23.9
63a0a3c05865b8927e89e9ad Diabetes mellitus (E10-E14) Diabetes Pennsylvania 3704 21.1
63a0a3c05865b8927e89e9ae Diabetes mellitus (E10-E14) Diabetes Rhode Island 275 18.8
63a0a3c05865b8927e89e9af Diabetes mellitus (E10-E14) Diabetes South Carolina 1535 24.5
63a0a

63a0a3c05865b8927e89ebcc Diabetes mellitus (E10-E14) Diabetes Florida 5782 19.5
63a0a3c05865b8927e89ebcd Diabetes mellitus (E10-E14) Diabetes Georgia 2238 21.2
63a0a3c05865b8927e89ebce Diabetes mellitus (E10-E14) Diabetes Hawaii 285 15.1
63a0a3c05865b8927e89ebcf Diabetes mellitus (E10-E14) Diabetes Idaho 375 19.8
63a0a3c05865b8927e89ebd0 Diabetes mellitus (E10-E14) Diabetes Illinois 2781 18.7
63a0a3c05865b8927e89ebd1 Diabetes mellitus (E10-E14) Diabetes Indiana 1992 26.0
63a0a3c05865b8927e89ebd2 Diabetes mellitus (E10-E14) Diabetes Iowa 847 20.9
63a0a3c05865b8927e89ebd3 Diabetes mellitus (E10-E14) Diabetes Kansas 727 21.2
63a0a3c05865b8927e89ebd4 Diabetes mellitus (E10-E14) Diabetes Kentucky 1479 28.4
63a0a3c05865b8927e89ebd5 Diabetes mellitus (E10-E14) Diabetes Louisiana 1324 25.5
63a0a3c05865b8927e89ebd6 Diabetes mellitus (E10-E14) Diabetes Maine 463 23.9
63a0a3c05865b8927e89ebd7 Diabetes mellitus (E10-E14) Diabetes Maryland 1358 19.6
63a0a3c05865b8927e89ebd8 Diabetes mellitus (E10-E

63a0a3c05865b8927e89ee0c Diabetes mellitus (E10-E14) Diabetes Illinois 2817 19.2
63a0a3c05865b8927e89ee0d Diabetes mellitus (E10-E14) Diabetes Indiana 2030 26.9
63a0a3c05865b8927e89ee0e Diabetes mellitus (E10-E14) Diabetes Iowa 1077 26.7
63a0a3c05865b8927e89ee0f Diabetes mellitus (E10-E14) Diabetes Kansas 684 20.2
63a0a3c05865b8927e89ee10 Diabetes mellitus (E10-E14) Diabetes Kentucky 1458 28.2
63a0a3c05865b8927e89ee11 Diabetes mellitus (E10-E14) Diabetes Louisiana 1199 23.5
63a0a3c05865b8927e89ee12 Diabetes mellitus (E10-E14) Diabetes Maine 407 21.9
63a0a3c05865b8927e89ee13 Diabetes mellitus (E10-E14) Diabetes Maryland 1243 18.3
63a0a3c05865b8927e89ee14 Diabetes mellitus (E10-E14) Diabetes Massachusetts 1398 16.7
63a0a3c05865b8927e89ee15 Diabetes mellitus (E10-E14) Diabetes Michigan 2751 22.5
63a0a3c05865b8927e89ee16 Diabetes mellitus (E10-E14) Diabetes Minnesota 1221 18.6
63a0a3c05865b8927e89ee17 Diabetes mellitus (E10-E14) Diabetes Mississippi 1092 32.4
63a0a3c05865b8927e89ee18 Diabe

63a0a3c05865b8927e89efc7 Alzheimer disease (G30) Alzheimer disease Rhode Island 403 25.9
63a0a3c05865b8927e89efc8 Alzheimer disease (G30) Alzheimer disease South Carolina 1938 37.4
63a0a3c05865b8927e89efc9 Alzheimer disease (G30) Alzheimer disease South Dakota 434 36.2
63a0a3c05865b8927e89efca Alzheimer disease (G30) Alzheimer disease Tennessee 2672 38.1
63a0a3c05865b8927e89efcb Alzheimer disease (G30) Alzheimer disease Texas 6772 30.0
63a0a3c05865b8927e89efcc Alzheimer disease (G30) Alzheimer disease United States 93541 25.4
63a0a3c05865b8927e89efcd Alzheimer disease (G30) Alzheimer disease Utah 584 26.7
63a0a3c05865b8927e89efce Alzheimer disease (G30) Alzheimer disease Vermont 266 31.9
63a0a3c05865b8927e89efcf Alzheimer disease (G30) Alzheimer disease Virginia 1775 20.8
63a0a3c05865b8927e89efd0 Alzheimer disease (G30) Alzheimer disease Washington 3344 43.6
63a0a3c05865b8927e89efd1 Alzheimer disease (G30) Alzheimer disease West Virginia 620 25.5
63a0a3c05865b8927e89efd2 Alzheimer dise

...

63a0a3c05865b8927e8a0b1c Diabetes mellitus (E10-E14) Diabetes Louisiana 1774 42.3
63a0a3c05865b8927e8a0b1d Diabetes mellitus (E10-E14) Diabetes Maine 404 27.1
63a0a3c05865b8927e8a0b1e Diabetes mellitus (E10-E14) Diabetes Maryland 1519 30.1
63a0a3c05865b8927e8a0b1f Diabetes mellitus (E10-E14) Diabetes Massachusetts 1423 20.4
63a0a3c05865b8927e8a0b20 Diabetes mellitus (E10-E14) Diabetes Michigan 2785 28.0
63a0a3c05865b8927e8a0b21 Diabetes mellitus (E10-E14) Diabetes Minnesota 1317 26.0
63a0a3c05865b8927e8a0b22 Diabetes mellitus (E10-E14) Diabetes Mississippi 671 24.4
63a0a3c05865b8927e8a0b23 Diabetes mellitus (E10-E14) Diabetes Missouri 1625 26.8
63a0a3c05865b8927e8a0b24 Diabetes mellitus (E10-E14) Diabetes Montana 210 21.1
63a0a3c05865b8927e8a0b25 Diabetes mellitus (E10-E14) Diabetes Nebraska 393 20.8
63a0a3c05865b8927e8a0b26 Diabetes mellitus (E10-E14) Diabetes Nevada 343 17.5
63a0a3c05865b8927e8a0b27 Diabetes mellitus (E10-E14) Diabetes New Hampshire 311 25.0
63a0a3c05865b8927e8a0b28 

63a0a3c05865b8927e8a0c7c All Causes All causes Colorado 28294 796.1
63a0a3c05865b8927e8a0c7d All Causes All causes Connecticut 29827 772.1
63a0a3c05865b8927e8a0c7e All Causes All causes Delaware 7112 903.1
63a0a3c05865b8927e8a0c7f All Causes All causes District of Columbia 5951 1049.9
63a0a3c05865b8927e8a0c80 All Causes All causes Florida 167269 806.1
63a0a3c05865b8927e8a0c81 All Causes All causes Georgia 64485 966.4
63a0a3c05865b8927e8a0c82 All Causes All causes Hawaii 8394 660.4
63a0a3c05865b8927e8a0c83 All Causes All causes Idaho 9753 803.5
63a0a3c05865b8927e8a0c84 All Causes All causes Illinois 105430 863.5
63a0a3c05865b8927e8a0c85 All Causes All causes Indiana 55198 912.7
63a0a3c05865b8927e8a0c86 All Causes All causes Iowa 27791 780.1
63a0a3c05865b8927e8a0c87 All Causes All causes Kansas 24647 844.4
63a0a3c05865b8927e8a0c88 All Causes All causes Kentucky 39861 997.1
63a0a3c05865b8927e8a0c89 All Causes All causes Louisiana 41757 1013.1
63a0a3c05865b8927e8a0c8a All Causes All causes

63a0a3c05865b8927e8a0da9 Diseases of heart (I00-I09,I11,I13,I20-I51) Heart disease Virginia 14913 234.5
63a0a3c05865b8927e8a0daa Diseases of heart (I00-I09,I11,I13,I20-I51) Heart disease Washington 11281 203.3
63a0a3c05865b8927e8a0dab Diseases of heart (I00-I09,I11,I13,I20-I51) Heart disease West Virginia 6325 298.8
63a0a3c05865b8927e8a0dac Diseases of heart (I00-I09,I11,I13,I20-I51) Heart disease Wisconsin 13023 224.3
63a0a3c05865b8927e8a0dad Diseases of heart (I00-I09,I11,I13,I20-I51) Heart disease Wyoming 985 211.9
63a0a3c05865b8927e8a0dae Influenza and pneumonia (J09-J18) Influenza and pneumonia Alabama 1105 24.9
63a0a3c05865b8927e8a0daf Influenza and pneumonia (J09-J18) Influenza and pneumonia Alaska 36 13.1
63a0a3c05865b8927e8a0db0 Influenza and pneumonia (J09-J18) Influenza and pneumonia Arizona 1133 22.6
63a0a3c05865b8927e8a0db1 Influenza and pneumonia (J09-J18) Influenza and pneumonia Arkansas 735 25.1
63a0a3c05865b8927e8a0db2 Influenza and pneumonia (J09-J18) Influenza and pn

63a0a3c05865b8927e8a0fae Diabetes mellitus (E10-E14) Diabetes United States 69301 25.1
63a0a3c05865b8927e8a0faf Diabetes mellitus (E10-E14) Diabetes Utah 530 34.8
63a0a3c05865b8927e8a0fb0 Diabetes mellitus (E10-E14) Diabetes Vermont 163 26.1
63a0a3c05865b8927e8a0fb1 Diabetes mellitus (E10-E14) Diabetes Virginia 1564 24.4
63a0a3c05865b8927e8a0fb2 Diabetes mellitus (E10-E14) Diabetes Washington 1331 24.5
63a0a3c05865b8927e8a0fb3 Diabetes mellitus (E10-E14) Diabetes West Virginia 759 35.9
63a0a3c05865b8927e8a0fb4 Diabetes mellitus (E10-E14) Diabetes Wisconsin 1309 23.3
63a0a3c05865b8927e8a0fb5 Diabetes mellitus (E10-E14) Diabetes Wyoming 111 24.1
63a0a3c05865b8927e8a0fb6 Diseases of heart (I00-I09,I11,I13,I20-I51) Heart disease Alabama 13406 299.5
63a0a3c05865b8927e8a0fb7 Diseases of heart (I00-I09,I11,I13,I20-I51) Heart disease Alaska 607 211.7
63a0a3c05865b8927e8a0fb8 Diseases of heart (I00-I09,I11,I13,I20-I51) Heart disease Arizona 10584 214.0
63a0a3c05865b8927e8a0fb9 Diseases of heart

63a0a3c05865b8927e8a11e5 Diabetes mellitus (E10-E14) Diabetes Rhode Island 236 20.1
63a0a3c05865b8927e8a11e6 Diabetes mellitus (E10-E14) Diabetes South Carolina 1118 29.6
63a0a3c05865b8927e8a11e7 Diabetes mellitus (E10-E14) Diabetes South Dakota 196 23.1
63a0a3c05865b8927e8a11e8 Diabetes mellitus (E10-E14) Diabetes Tennessee 1436 26.1
63a0a3c05865b8927e8a11e9 Diabetes mellitus (E10-E14) Diabetes Texas 4931 29.9
63a0a3c05865b8927e8a11ea Diabetes mellitus (E10-E14) Diabetes United States 68399 25.0
63a0a3c05865b8927e8a11eb Diabetes mellitus (E10-E14) Diabetes Utah 472 31.4
63a0a3c05865b8927e8a11ec Diabetes mellitus (E10-E14) Diabetes Vermont 179 29.3
63a0a3c05865b8927e8a11ed Diabetes mellitus (E10-E14) Diabetes Virginia 1486 23.6
63a0a3c05865b8927e8a11ee Diabetes mellitus (E10-E14) Diabetes Washington 1307 24.5
63a0a3c05865b8927e8a11ef Diabetes mellitus (E10-E14) Diabetes West Virginia 734 34.8
63a0a3c05865b8927e8a11f0 Diabetes mellitus (E10-E14) Diabetes Wisconsin 1273 22.9
63a0a3c05865

In [105]:
# count=0
# for row in brfss_df.itertuples():
#     count=count+1
#     #print(row)
#     #print(str(count)+"==>"+str(row))
#     print(row[1],row[2],row[3],row[4],row[5],row[6],row[7])
#     cursor.execute("INSERT INTO public.brfss(_id, year, state, yes, no, category, condition)  VALUES ('"+str(row[1])+"','"+str(row[2])+"','"+str(row[3])+"','"+str(row[4])+"','"+str(row[5])+"','"+str(row[6])+"','"+str(row[7])+"');")

In [106]:
count

10868

In [52]:
conn.commit()

In [53]:
conn.close()